# 记录一些carla的基本操作

In [ ]:
import carla
from numpy import random

#### 1.连接到客户端并检索世界对象

In [ ]:
client = carla.Client('localhost', 2000)
world = client.get_world()

#### 2.打印能够使用的地图

In [ ]:
print(f'够使用的地图{client.get_available_maps()}')

#### 3.加载新地图

In [ ]:
client.load_world('Town07')

#### 4.重新加载当前地图并重置状态

client.reload_world()

#### 5.获取对象信息

In [ ]:
print(f'所有对象的名字{world.get_names_of_all_objects()}')
print(f'建筑物名称列表{filter(lambda x: "Building" in x, world.get_names_of_all_objects())}')
print(f'获取所有参与者的列表，例如车辆和行人:{world.get_actors()}')
print(f'筛选列表以查找车辆{world.get_actors().filter("*vehicle*")}')
print(f'当前的天气是:{world.get_weather()}')

#### 6.生成一个车辆

In [ ]:
# 获取蓝图库并筛选车辆蓝图
vehicle_bps = world.get_blueprint_library().filter('*vehicle*')

# 打印所有可用的蓝图
# for actor in world.get_blueprint_library():
#     print(actor)

# 打印所有可用的车辆蓝图
# for actor in world.get_blueprint_library().filter('vehicle'):
#     print(actor)
#
# vehicle_blueprint = world.get_blueprint_library().find('vehicle.audi.tt')

# 随机选择一个载具蓝图生成
vehicle_bp = random.choice(vehicle_bps)

# 我们需要一个可以正常生成载具的地方，所以我们将使用地图上预设的生成点，并随机选择一个。
spawn_point = random.choice(world.get_map().get_spawn_points())

# 现在我们来生成车辆。
world.spawn_actor(vehicle_bp, spawn_point)

vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)

#### 7.观察者的一些设置

In [ ]:
# 检索旁观者对象
spectator = world.get_spectator()
# 通过变换获取观众的位置和旋转。
transform = spectator.get_transform()
# 位置信息
print(f'观察者位置信息:{transform}')
location = transform.location
rotation = transform.rotation

# 将旁观者设置为空变换。
spectator.set_transform(carla.Transform())
# 这将把观众置于地图的原点，俯仰角、偏航角和横滚角均为 0 度——这是在地图上确定方向的好方法。

# 旁观者所在位置生成一辆载具
vehicle = world.try_spawn_actor(vehicle_bp, spectator.get_transform())

#### 8.摄像机的一些设置

In [ ]:
# 创建一个变换，将摄像机放置在车辆顶部。
camera_init_trans = carla.Transform(carla.Location(z=1.5))

# 通过定义相机属性的蓝图来创建相机。
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

# 我们生成摄像头并将其安装到我们的自驱车辆上。
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
# 例如：手动 tick 或设置自动
for _ in range(100):
    world.tick()
    # 使用 PyGame 回调函数启动相机
    camera.listen(lambda image: image.save_to_disk('/home/jiangchengxuan/PycharmProjects/carla-RL/out/%06d.png' % image.frame))

#### 9.生成多个载具

In [ ]:
# 在地图上随机生成 50 辆载具
for i in range(0,50):
    vehicle_bps = world.get_blueprint_library().filter('*vehicle*')
    vehicle_bp = random.choice(vehicle_bps)
    spawn_point = random.choice(world.get_map().get_spawn_points())
    vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)


# 将生成点可视化在地图上。
spawn_points = world.get_map().get_spawn_points()
for i, spawn_point in enumerate(spawn_points):
    # 旁观者窗口中显示的原始生成点索引
    world.debug.draw_string(spawn_point.location, str(i), life_time=100)
    # 我们还可以画一个箭头来查看生成点的方向。（即车辆生成时朝向哪个方向）
    world.debug.draw_arrow(spawn_point.location, spawn_point.location + spawn_point.get_forward_vector(), life_time=100)

vehicle_bps = world.get_blueprint_library().filter('*vehicle*')
# 现在我们可以记下我们感兴趣的生成点索引，并在这条街道上放置车辆
for ind in range(0, 100):
    world.try_spawn_actor(random.choice(vehicle_bps), random.choice(spawn_points))

#### 10. 同步和异步模式区别
    异步模式下，CARLA 服务器会尽可能快地运行，客户端请求会实时处理。而在同步模式下，运行 Python 代码的客户端会接管控制权，并告知服务器何时更新。
    如果您正在进行实验或设置模拟程序，异步模式是运行 CARLA 的合适模式，这样您就可以在放置角色时与旁观者一起在地图上自由移动。
    当您想要开始生成训练数据或在模拟程序中部署智能体时，建议您使用同步模式，因为这样可以为您提供更好的控制和可预测性。
##### 10.1 设置方法

In [ ]:
# 如果启用了同步模式，并且正在运行traffic_manager，则也必须将其设置为同步模式。
# 请阅读此文档了解具体操作方法。https://carla.readthedocs.io/en/0.9.16/adv_traffic_manager/#synchronous-mode
settings = world.get_settings()
settings.synchronous_mode = True # 启用同步模式
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

##### 10.2 如何禁用同步模式
    要禁用同步模式，只需将变量设置为 False 或使
    用脚本 PythonAPI/util/config.py（此脚本无法启用同步模式，只能禁用。启用同步模式后，服务器会等待客户端的 tick 请求。使用此脚本，用户将无法按需发送 tick 请求。）

##### 10.3 同步模式的注意事项
    对于运行速度较慢的客户端应用程序，以及需要在不同元素（例如传感器）之间进行同步的情况，同步模式尤为重要。
    如果客户端速度过慢，而服务器又没有及时响应，就会发生信息溢出。客户端将无法处理所有数据，导致数据丢失或混淆。
    同样，如果传感器数量众多且采用异步方式，则无法确定所有传感器是否都在使用模拟过程中同一时刻的数据。
    以下代码片段是对前一段代码的扩展。客户端创建一个摄像头传感器，将当前步骤的图像数据存储在一个队列中，并在从队列中检索数据后向服务器发送通知。
    关于涉及多个传感器的更复杂示例，请参见此处https://github.com/carla-simulator/carla/blob/master/PythonAPI/examples/synchronous_mode.py。
    来自基于GPU的传感器（主要是摄像头）的数据通常存在几帧的延迟。因此，同步至关重要。

In [ ]:
import queue
# 获取相机蓝图
vehicle_bps = world.get_blueprint_library().filter('*vehicle*')
# 初始队列
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)

camera = world.spawn_actor(vehicle_bps, transform)
image_queue = queue.Queue()
camera.listen(image_queue.put)

while True:
    world.tick()
    image = image_queue.get()
    break


##### 10.4 异步模式
    世界拥有异步方法，可以让客户端等待服务器的时钟滴答声，或者在收到滴答声时执行某些操作。

In [ ]:
# 世界拥有“异步”方法，可以让客户端等待服务器的时钟滴答声，或者在收到滴答声时执行某些操作。
# 等待下一次数据滴答，并获取该滴答的快照。
world_snapshot = world.wait_for_tick()

# 注册一个回调函数，以便在每次收到新快照时调用。
world.on_tick(lambda world_snapshot: do_something(world_snapshot))

#### 11.日志记录
    记录器可以将重现先前模拟所需的所有数据保存到一个文件中。这些数据包括车辆的位置和速度、交通信号灯的状态、行人的位置和速度以及太阳的位置和天气状况等详细信息。
    数据会被记录到一个二进制文件中，稍后 Carla 服务器可以加载该文件以精确地重现模拟。
    角色会根据记录文件中包含的数据在每一帧进行更新。当前模拟中出现在记录中的角色将被移动或重新生成以模拟记录。未出现在记录中的角色将继续执行它们的任务，仿佛什么都没发生过。
    记录文件包含有关多种元素的信息。
    Actors——创建和销毁、边界框和触发框。
    Traffic lights——状态变化和时间设置。
    Vehicles——位置和方向、线速度和角速度、灯光状态以及物理控制。
    Pedestrians——位置和方向以及线速度和角速度。
    Lights——建筑物、街道和车辆的灯光状态。

In [ ]:
# 能够记录世界变化的日志，后续能够恢复世界的运行（据估计，录制 1 小时，包含 50 个交通信号灯和 100 辆车的视频，大约需要 200MB 的文件大小。）
client.start_recorder("/home/carla/recording01.log")

# 如果要额外记录车辆和行人的线速度和角速度、交通信号灯时间设置、执行时间、角色的触发器和边界框，以及车辆的物理控制
client.start_recorder("/home/carla/recording01.log", True)

# 停止记录操作
client.stop_recorder()

# 回放操作
# param:start 记录模拟开始时间（以秒为单位）。
# param:duration 回放几秒，0代表从头开始，回放结束后所有车开始自动驾驶
# param:camera 摄像机将聚焦的actor_id 0代表视角能自由移动
client.replay_file("recording01.log", start, duration, camera)

#### carla的启动
    高画质模式启动carla
    ./CarlaUE4.sh -quality-level=Epic

    低画质启动carla
    ./CarlaUE4.sh -quality-level=Low